# This example is a duplication of [Mason Victor](http://www.recursionpharma.com/mason.html)'s [lightning talk at SciPy 2016](https://youtu.be/sv9S-25XKe4?t=54m40s) 

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import bokeh.plotting as bop
from bokeh.models import ColumnDataSource, HoverTool, WheelZoomTool
bop.output_notebook()

Loading BokehJS ...

In [3]:
import keras
import theano
from keras.datasets import mnist as mnist_data
from keras.layers import Input
from keras.layers.convolutional import (
    Convolution2D, ZeroPadding2D,
    AveragePooling2D, MaxPooling2D
)
from keras.layers.core import Dense, Activation, Flatten, Lambda, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.models import Model
from keras import backend as K
from keras.utils.np_utils import to_categorical

theano.config.compute_test_value = 'off'

Using TensorFlow backend.
Using gpu device 0: GeForce GTX 750 Ti (CNMeM is enabled with initial size: 65.0% of memory, cuDNN 5110)
/home/schowell/data/myPrograms/anaconda3/envs/k35/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


# Traditional Dimensionality Reduction

## PCA

In [4]:
from sklearn.decomposition import PCA

In [5]:
(X_train, y_train), (X_test, y_test) = mnist_data.load_data()

X_test = np.reshape(X_test, (10000, 784))

X_test_df = pd.DataFrame(X_test)
X_test_df = X_test_df[X_test_df.columns[X_test_df.std() > 0.0]]
X_test_df = (X_test_df - X_test_df.mean()) / X_test_df.std()
X_test = X_test_df.values

pcaMod = PCA(n_components=2)
representations=pcaMod.fit_transform(X_test)

In [6]:
from bokeh.palettes import Category10_10 as palette

In [7]:
representations.shape

(10000, 2)

In [8]:
img_urls = ['MNIST_images/img%d.jpg' % x for x in range(len(X_test))]

In [9]:
color_map = dict(zip(np.arange(10), palette))
colors = [color_map[i] for i in y_test]
img_urls = ['MNIST_images/img%d.jpg' % x for x in range(len(X_test))]

cds = ColumnDataSource(data=dict(x=representations[:, 0],
                                 y=representations[:, 1],
                                 color=colors,
                                 img_url=img_urls))

hover = HoverTool(tooltips="""<img src="@img_url" heigh="42" wiwdth="42"></img>""")
p = bop.figure(tools=[hover])
p.circle(x='x', y='y', size=5, alpha=0.5, color='color', source=cds)
bop.show(p)

# Deep Learning Dimensionality Reduction
![title](https://www.researchgate.net/profile/Luis_Ferraz/publication/269935583/figure/fig1/AS:294994507255823@1447343579348/Figure-1-Left-Schematic-of-a-siamese-network-where-pairs-of-input-patches-are.png)
(see https://www.researchgate.net/figure/269935583_fig1_Figure-1-Left-Schematic-of-a-siamese-network-where-pairs-of-input-patches-are.png)

In [10]:
def contrastive_loss(y_true, y_pred):
    '''
    Contrastive loss from Hadsell et al. 2006
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1.0
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [11]:
def create_base_network(input_dim=(1, 28, 28), 
                        final_dim=2):
    inp = Input(shape=input_dim)
    # input shape = 28 x 28
    x = Convolution2D(32, nb_row=3, 
                      nb_col=3,
                      border_mode='valid')(inp)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)
    x = MaxPooling2D((2, 2))(x)
    
    x = Convolution2D(32, nb_row=3, nb_col=3)(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)
    x = MaxPooling2D((2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(128)(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    
    x = Dense(final_dim)(x)
    network = Model(inp, x)
    return network

In [12]:
def create_model(metric, metric_shape, final_dim=2):
    base_network = create_base_network(final_dim=final_dim)
    input_a = Input(shape=(1, 28, 28), name='left')
    input_b = Input(shape=(1, 28, 28), name='right')
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    distance = Lambda(metric, output_shape=metric_shape, name='similarity')([processed_a, processed_b])
    
    model = Model(input=[input_a, input_b], output=distance)
    
    model.compile(loss=contrastive_loss, optimizer='adam')
    return model

In [36]:
def create_siam_minibatch(input_sets, input_set_lens, batch_size=32):
    left = []
    left_classes = []
    right = []
    right_classes = []
    sims = []
    for i in range(batch_size):
        if (i % 2) == 0:
            left_class = np.random.choice(list(input_sets.keys()), 1)[0]  #sh cast keys as list, Python 2->3
            right_class = left_class
            sims.append(1)
        else:
            left_class, right_class = np.random.choice(list(input_sets.keys()), 2, replace=False)
            sims.append(0)
        left_classes.append(left_class)
        right_classes.append(right_class)
        left.append(input_sets[left_class][np.random.randint(input_set_lens[left_class])])
        right.append(input_sets[right_class][np.random.randint(input_set_lens[right_class])])
        inp = {'left': np.array(left, dtype=np.float32),
               'right': np.array(right, dtype=np.float32)}
        out = {'similarity': np.array(sims)}
        
        return inp, out

In [14]:
def siam_generator(input_sets, input_set_lens):
    while True:
        yield create_siam_minibatch(input_sets, input_set_lens)

In [15]:
(X_train, y_train), (X_test, y_test) = mnist_data.load_data()

X_train = X_train[:, np.newaxis, :, :] / 255.
x_test = X_test[:, np.newaxis, :, :] / 255.

y_train_ser = pd.Series(y_train)
y_test_ser = pd.Series(y_test)

X_train_sets = {y: X_train[list(y_train_ser[y_train_ser == y].index)]
                for y in range(10)}
X_test_sets = {y: X_test[list(y_test_ser[y_test_ser == y].index)] 
               for y in range(10)}

#sh updated from `iteritems` to `items`, Python 2->3
X_train_lens = {k: len(v) for k, v in X_train_sets.items()}  
X_test_lens = {k: len(v) for k, v in X_test_sets.items()}

In [20]:
def create_plot(metric, metric_shape):
    train_gen = siam_generator(X_train_sets,
                               X_train_lens)
    val_gen = siam_generator(X_test_sets, 
                             X_test_lens)
    
    model = create_model(metric=metric, metric_shape=metric_shape)
    model.fit_generator(train_gen, 8000, 25, validation_data=val_gen, nb_val_samples=3200)
    
    vision_model = model.layers[2]
    
    img_input = Input(shape=(1, 28, 28))
    processed = vision_model(img_input)
    single_track_model = Model(input=img_input, output=processed)
    
    single_track_model.compile(optimizer='adam', loss=contrastive_loss)
    
    representations = single_track_model.predict(X_test)
    
    color_map = dict(zip(np.arange(10), palette))
    colors = [color_map[i] for i in y_test]
    img_urls = ['MNIST_images/img%d.jpg' % x for x in range(len(X_test))]
    
    cds = ColomnDataSource(data=dict(x=representations[:, 0],
                                     y=representations[:, 1],
                                     color=colors,
                                     img_url=img_urls))

    hover = HoverTool(tooltips="""<img src="@img_url" heigh="42" wiwdth="42"></img>""")
    p = bop.figure(tools=[hover])
    p.circle(x='x', y='y', size=5, alpha=0.5, color='color', source=cds)
    bop.show(p)

# Euclidean Distance
------------------------------------------------------
$$d(x, y) = \sqrt{\sum_i(x_i - y_i)^2}$$

In [21]:
def euclidean_distance(vects):
    #sh why not use `scipy.spatial.distance.cdist` for this?
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

In [22]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

create_plot(euclidean_distance, eucl_dist_output_shape)